<a href="https://colab.research.google.com/github/cahallsherr/LIGN168/blob/main/finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
2# Step 1: install essential libraries
!pip3 install librosa
!pip install pydub
!pip install torch torchvision torchaudio
!pip install datasets transformers
!pip install praatio librosa
!pip install g2p_en g2pM
!pip install sounddevice
!pip install wavio
!pip install scipy
!apt-get update
!apt-get install libsndfile1

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import os
from os.path import exists, join, expanduser

os.chdir(expanduser("~"))
charsiu_dir = 'charsiu'
if exists(charsiu_dir):
  !rm -rf /root/charsiu
if not exists(charsiu_dir):
  ! git clone -b development https://github.com/lingjzhu/$charsiu_dir
  ! cd charsiu && git checkout && cd -

os.chdir(charsiu_dir)

Cloning into 'charsiu'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 686 (delta 110), reused 79 (delta 79), pack-reused 555
Receiving objects: 100% (686/686), 1.20 MiB | 7.06 MiB/s, done.
Resolving deltas: 100% (364/364), done.
Your branch is up to date with 'origin/development'.
/root


In [ ]:
# Step 3: import essential libraries
import sys
import torch
from itertools import groupby

sys.path.insert(0,'src')

In [ ]:
# [optional step:] use your own data from your google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#pretrained model. This one is textless

from Charsiu import charsiu_predictive_aligner

#initialize model

charsiu = charsiu_predictive_aligner(aligner='charsiu/en_w2v2_fc_10ms')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/403 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at charsiu/en_w2v2_fc_10ms were not used when initializing Wav2Vec2ForFrameClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForFrameClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForFrameClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForFrameClassification were not initialized from the model checkpoint at charsiu/en_w2v2_fc_10ms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.

In [ ]:
from pydub import AudioSegment

def give_me_music(wavfile_name):

    uploaded_wav = AudioSegment.from_file(wavfile_name)
    resampled_wav = uploaded_wav.set_frame_rate(16000)
    resampled_wav.export(wavfile_name, format="wav")

    segments_list = charsiu.align(audio = wavfile_name)
    instrumental_audio = []
    phoneme_list = []
    sound_list = []
    final_audio = AudioSegment.silent(duration=0)

    phoneme_dict = {'UW': 'high back vowel','IY': 'high front vowel','IH': 'high front vowel','EY': 'mid front vowel',
                    'AA': 'low front vowel', 'AY': 'low front vowel', 'AW': 'low back vowel','AO': 'low back vowel', 'AE': 'low front vowel',
                    'UH': 'high back vowel','OY': 'mid back vowel','OW': 'mid back vowel', 'ER': 'mid front vowel','EH': 'mid front vowel','AH': 'mid back vowel',
                    'T': 'voiceless stop','P': 'voiceless stop', 'K': 'voiceless stop','G': 'voiced stop','D': 'voiced stop', 'B': 'voiced stop',
                    'NG': 'nasal','N': 'nasal','M': 'nasal',
                    'SH': 'voiceless fricative', 'DH': 'voiced fricative', 'F': 'voiceless fricative','S': 'voiceless fricative', 'TH': 'voiceless fricative',
                    'V': 'voiced fricative', 'Z': 'voiced fricative', 'ZH': 'voiced fricative','HH': 'voiceless fricative',
                    'CH': 'voiceless affricate', 'JH': 'voiced affricate',
                    'Y': 'approximant','W': 'approximant','R': 'approximant','L': 'approximant',
                    '[SIL]': 'silence'}

    instrument_dict = {
                   'high back vowel' :'/content/drive/MyDrive/Colab Notebooks/highbackvowel.mp3',
                   'high front vowel':'/content/drive/MyDrive/Colab Notebooks/highfrontvowel.mp3',
                   'low back vowel': '/content/drive/MyDrive/Colab Notebooks/lowbackvowel.mp3',
                   'low front vowel': '/content/drive/MyDrive/Colab Notebooks/lowfrontvowel.mp3',
                   'mid back vowel': '/content/drive/MyDrive/Colab Notebooks/midbackvowel.mp3',
                   'mid front vowel': '/content/drive/MyDrive/Colab Notebooks/midfrontvowel.mp3',
                   'voiced stop': '/content/drive/MyDrive/Colab Notebooks/voicedstop.mp3' ,
                   'voiceless stop':'/content/drive/MyDrive/Colab Notebooks/voicelessstop.mp3',
                   'nasal': '/content/drive/MyDrive/Colab Notebooks/nasal.mp3' ,
                   'voiced fricative': '/content/drive/MyDrive/Colab Notebooks/voicedfricative.mp3',
                   'voiceless fricative': '/content/drive/MyDrive/Colab Notebooks/voicedfricative.mp3',
                   'voiced affricate': '/content/drive/MyDrive/Colab Notebooks/voicedaffricate.mp3',
                   'voiceless affricate': '/content/drive/MyDrive/Colab Notebooks/voicelessaffricate.mp3' ,
                   'approximant': '/content/drive/MyDrive/Colab Notebooks/approximant.mp3',
                   'silence': '/content/drive/MyDrive/Colab Notebooks/silence.mp3'}

    for each_segment in segments_list:
      segment_labels = [segment_tuple[2] for segment_tuple in segments_list]

    for each_segment in segment_labels:
        phoneme_list.append(each_segment)

    for each_phoneme in phoneme_list:
        selected_sound = phoneme_dict[each_phoneme]
        sound_list.append(selected_sound)

    for each_sound in sound_list: # i want this to take the list of types and give mp3s
        if each_sound in instrument_dict: # how to get mp3
            selected_instrument = instrument_dict[each_sound]  #will give me value? from dict
            instrumental_audio.append(selected_instrument) #can add just value to the audioclips list

    for mp3_file, segment in zip(instrumental_audio, segments_list):
      audio = AudioSegment.from_mp3(mp3_file)
      duration = segment[1] - segment[0]
      trimmed_audio = audio[50:int(duration*1000)]
      final_audio += trimmed_audio

    return final_audio


In [ ]:
#for speech sounds!
from pydub import AudioSegment

def speech_to_speech(wavfile_name):

    uploaded_wav = AudioSegment.from_file(wavfile_name)
    resampled_wav = uploaded_wav.set_frame_rate(16000)
    resampled_wav.export(wavfile_name, format="wav")

    segments_list2 = charsiu.align(audio = wavfile_name)
    changed_audio = []
    phoneme_list2 = []
    sound_list2 = []
    final_audio2 = AudioSegment.silent(duration=0)

    phoneme_dict2 = {'UW': 'high vowel','IY': 'high vowel','IH': 'high vowel','EY': 'mid vowel',
                    'AA': 'low vowel', 'AY': 'low vowel', 'AW': 'low vowel','AO': 'low vowel', 'AE': 'low vowel',
                    'UH': 'high vowel','OY': 'mid vowel','OW': 'mid vowel', 'ER': 'mid vowel','EH': 'mid vowel','AH': 'mid vowel',
                    'T': 'stop','P': 'stop', 'K': 'stop','G': 'stop','D': 'stop', 'B': 'stop',
                    'NG': 'nasal','N': 'nasal','M': 'nasal',
                    'SH': 'fricative', 'DH': 'fricative', 'F': 'fricative','S': 'fricative', 'TH': 'fricative',
                    'V': 'fricative', 'Z': 'fricative', 'ZH': 'fricative','HH': 'fricative',
                    'CH': 'affricate', 'JH': 'affricate',
                    'Y': 'approximant','W': 'approximant','R': 'approximant','L': 'approximant',
                    '[SIL]': 'silence'}

    speech_sound_dict = {
                   'high vowel' :'/content/drive/MyDrive/Colab Notebooks/highvowel2.mp3',
                   'low vowel': '/content/drive/MyDrive/Colab Notebooks/lowvowel2.mp3',
                   'mid vowel': '/content/drive/MyDrive/Colab Notebooks/midvowel2.mp3',
                   'stop': '/content/drive/MyDrive/Colab Notebooks/stop2.mp3',
                   'nasal': '/content/drive/MyDrive/Colab Notebooks/nasal2.mp3' ,
                   'fricative': '/content/drive/MyDrive/Colab Notebooks/fricative2.mp3',
                   'affricate': '/content/drive/MyDrive/Colab Notebooks/affricate2.mp3',
                   'approximant': '/content/drive/MyDrive/Colab Notebooks/approximant2.mp3',
                   'silence': '/content/drive/MyDrive/Colab Notebooks/silence.mp3'}

    for each_segment in segments_list2:
      segment_labels2 = [segment_tuple[2] for segment_tuple in segments_list2]

    for each_segment in segment_labels2:
        phoneme_list2.append(each_segment)

    for each_phoneme in phoneme_list2:
        selected_sound = phoneme_dict2[each_phoneme]
        sound_list2.append(selected_sound)

    for each_sound in sound_list2:
        if each_sound in speech_sound_dict:
            selected_speech_sound = speech_sound_dict[each_sound]
            changed_audio.append(selected_speech_sound)

    for mp3_file, segment in zip(changed_audio, segments_list2):
      audio = AudioSegment.from_mp3(mp3_file)
      duration = segment[1] - segment[0]
      trimmed_audio = audio[50:int(duration*1000)]
      final_audio2 += trimmed_audio

    return final_audio2

In [ ]:
@article{zhu2022charsiu,
  title={Phone-to-audio alignment without text: A Semi-supervised Approach},
  author={Zhu, Jian and Zhang, Cong and Jurgens, David},
  journal={IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)},
  year={2022}
 }

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-15-bb5c1af65c51>, line 2)